# Debugging Your Code

## Why Debugging Matters

Every programmer spends significant time debugging. Studies suggest that developers spend 35-50% of their time finding and fixing bugs. For statistical computing and scientific software, debugging is particularly important:

* *Correctness:* A subtle bug in a statistical algorithm can produce plausible but incorrect results. Unlike a crash, wrong numerical output might go unnoticed until it affects downstream analyses or publications.

* *Complexity:* Scientific code often involves intricate mathematical operations, edge cases in numerical computation, and interactions between multiple components.

* *Reproducibility:* Bugs that appear intermittently or depend on specific data characteristics are especially problematic for reproducible research.

Debugging is a skill that improves with practice. Experienced programmers don't write bug-free code; they've learned systematic approaches to find and fix bugs efficiently.

### Types of Bugs

Understanding the different types of bugs helps you choose appropriate debugging strategies:

**Syntax errors** are caught by Python before your code runs. These include typos, missing colons, and unmatched parentheses.

In [ ]:
# SyntaxError: expected ':'
def calculate_mean(data)
    return sum(data) / len(data)

**Runtime errors** occur when code executes but encounters an impossible operation. These produce exceptions with tracebacks.

In [ ]:
def calculate_mean(data):
    return sum(data) / len(data)

calculate_mean([])  # ZeroDivisionError: division by zero

**Logic errors** are the hardest to find. The code runs without errors but produces incorrect results.

In [ ]:
def calculate_variance(data):
    mean = sum(data) / len(data)
    # Bug: should be (x - mean)**2, not (x - mean)
    squared_diffs = [(x - mean) for x in data]
    return sum(squared_diffs) / len(data)

This function runs without errors but returns the wrong value. Logic errors require understanding what the code *should* do, not just what it *does*.

### Question

Classify each of the following as a syntax error, runtime error, or logic error:

1. A function that calculates the median but returns the mean instead.

2. Code that tries to access `my_list[10]` when the list has only 5 elements.

3. A for loop written as `for i in range(10)` without a colon at the end.

4. A function that computes correlation but returns NaN for perfectly correlated data due to floating-point issues.

**Answer:**

1. **Logic error** - The code runs without exceptions but produces incorrect output. The programmer confused median with mean.

2. **Runtime error** - This raises an `IndexError` when the code tries to access an index that doesn't exist.

3. **Syntax error** - Python's parser catches the missing colon before any code executes.

4. **Logic error** - The function runs and returns a value (NaN), but this isn't the correct mathematical result. It might also be considered a numerical bug, which is a specific type of logic error common in scientific computing.

## Reading Error Messages

When Python encounters an error, it produces a traceback showing where the error occurred and what went wrong. Learning to read tracebacks is fundamental to debugging.

### Anatomy of a Traceback

Consider this code with a bug:

In [6]:
def normalize(data):
    mean = sum(data) / len(data)
    std = (sum((x - mean)**2 for x in data) / len(data)) ** 0.5
    return [(x - mean) / std for x in data]

def process_dataset(datasets):
    results = []
    for dataset in datasets:
        results.append(normalize(dataset))
    return results

data = [[1, 2, 3], [], [4, 5, 6]]  # Empty list causes ZeroDivisionError
process_dataset(data)

ZeroDivisionError: division by zero

Running this produces:

```
Traceback (most recent call last):
  File "example.py", line 13, in <module>
    process_dataset(data)
  File "example.py", line 9, in process_dataset
    results.append(normalize(dataset))
  File "example.py", line 2, in normalize
    mean = sum(data) / len(data)
ZeroDivisionError: division by zero
```

Read tracebacks from **bottom to top**:

1. The bottom line shows the actual error: `ZeroDivisionError: division by zero`
2. Above that, you see the chain of function calls that led to the error
3. Each frame shows the file, line number, function name, and the code that was executing

In this example, the issue is that `len([])` returns 0, causing division by zero. The traceback shows the error occurred in `normalize`, which was called from `process_dataset`, which was called from the main script.

### Common Error Types

**TypeError** occurs when an operation receives an object of the wrong type:

In [7]:
len(42)

TypeError: object of type 'int' has no len()

**ValueError** occurs when the type is correct but the value is inappropriate:

In [ ]:
int("hello")

**IndexError** occurs when accessing a sequence with an invalid index:

In [ ]:
my_list = [1, 2, 3]
my_list[5]

**KeyError** occurs when accessing a dictionary with a missing key:

In [ ]:
my_dict = {"a": 1, "b": 2}
my_dict["c"]

**AttributeError** occurs when accessing an attribute that doesn't exist:

In [ ]:
my_list = [1, 2, 3]
my_list.append(4)  # Works
my_list.add(5)     # Lists don't have an add method

### Question

The following traceback was produced by running a data analysis script. Read the traceback and answer the questions below.

```
Traceback (most recent call last):
  File "analysis.py", line 45, in <module>
    results = run_analysis(patient_data)
  File "analysis.py", line 32, in run_analysis
    stats = compute_statistics(group)
  File "analysis.py", line 18, in compute_statistics
    ci = confidence_interval(values, confidence=0.95)
  File "stats_utils.py", line 7, in confidence_interval
    std_err = np.std(data) / np.sqrt(len(data))
TypeError: object of type 'NoneType' has no len()
```

1. In which file and function did the error actually occur?
2. What was the chain of function calls that led to this error?
3. What is the likely cause of the error?

**Answer:**

1. The error occurred in `stats_utils.py`, line 7, in the `confidence_interval` function.

2. The call chain was: `analysis.py` main code called `run_analysis`, which called `compute_statistics`, which called `confidence_interval`.

3. The error is a `TypeError` saying `'NoneType' has no len()`. This means `data` is `None` when `confidence_interval` tries to compute `len(data)`. The bug is likely upstream: either `values` was `None` when passed to `confidence_interval`, or perhaps `compute_statistics` received a `group` that was `None`. You would need to check why `None` is being passed instead of actual data.

## Print Debugging

The simplest debugging technique is adding print statements to understand what your code is doing. While basic, strategic print debugging is often the fastest way to locate a bug.

### Basic Print Debugging

When your code produces unexpected output, add print statements to trace the execution:

In [ ]:
def calculate_weighted_mean(values, weights):
    print(f"Input values: {values}")
    print(f"Input weights: {weights}")

    total = 0
    weight_sum = 0
    for v, w in zip(values, weights):
        print(f"  Processing: value={v}, weight={w}")
        total += v * w
        weight_sum += w
        print(f"  Running total={total}, weight_sum={weight_sum}")

    result = total / weight_sum
    print(f"Final result: {result}")
    return result

calculate_weighted_mean([10, 20, 30], [1, 2, 3])

This reveals the intermediate state at each step, making it easy to spot where calculations go wrong.

### Printing Variable State

For numerical code, printing variable types and shapes is particularly useful:

In [ ]:
import numpy as np

def debug_matrix_multiply(A, B):
    print(f"A: type={type(A)}, shape={A.shape}, dtype={A.dtype}")
    print(f"B: type={type(B)}, shape={B.shape}, dtype={B.dtype}")

    result = A @ B
    print(f"Result: shape={result.shape}")
    return result

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
debug_matrix_multiply(A, B)

Shape mismatches are a common source of bugs in numerical code. Printing shapes helps catch them quickly.

### Conditional Printing

When debugging loops that run many times, print only when something interesting happens:

In [ ]:
def find_convergence(func, x0, tol=1e-6, max_iter=1000):
    x = x0
    for i in range(max_iter):
        x_new = func(x)
        diff = abs(x_new - x)

        # Only print every 100 iterations or when close to converging
        if i % 100 == 0 or diff < tol * 10:
            print(f"Iteration {i}: x={x_new:.6f}, diff={diff:.2e}")

        if diff < tol:
            print(f"Converged at iteration {i}")
            return x_new
        x = x_new

    print("Did not converge")
    return x

# Find fixed point of cos(x): x = cos(x)
import math
find_convergence(math.cos, 0.5)

### Limitations of Print Debugging

Print debugging has drawbacks:

* You must modify your code and remember to remove the prints later
* For complex bugs, you may need many print statements
* It doesn't let you interactively explore program state
* Printing inside tight loops can slow execution significantly

For simple bugs, print debugging is fast and effective. For complex bugs, consider the logging module or an interactive debugger.

### Question

The following function is supposed to compute the running maximum of an array, but it's returning incorrect results. Add print statements to help identify the bug, then fix it.

In [8]:
import numpy as np

def running_max(arr):
    """Return array where element i is the maximum of arr[0:i+1]."""
    result = np.zeros_like(arr)
    current_max = arr[0]
    for i in range(len(arr)):
        if arr[i] > current_max:
            current_max = arr[i]
            result[i] = current_max
    return result

# Test
test = np.array([3, 1, 4, 1, 5, 9, 2, 6])
print(running_max(test))
# Expected: [3, 3, 4, 4, 5, 9, 9, 9]
# Actual:   [0, 0, 4, 0, 5, 9, 0, 0]

[0 0 4 0 5 9 0 0]


**Answer:**

Adding print statements:

In [9]:
def running_max(arr):
    result = np.zeros_like(arr)
    current_max = arr[0]
    print(f"Initial current_max: {current_max}")
    for i in range(len(arr)):
        print(f"i={i}, arr[i]={arr[i]}, current_max={current_max}")
        if arr[i] > current_max:
            current_max = arr[i]
            result[i] = current_max
            print(f"  Updated: current_max={current_max}, result[{i}]={result[i]}")
        print(f"  result so far: {result}")
    return result

test = np.array([3, 1, 4, 1, 5, 9, 2, 6])
print(running_max(test))

Initial current_max: 3
i=0, arr[i]=3, current_max=3
  result so far: [0 0 0 0 0 0 0 0]
i=1, arr[i]=1, current_max=3
  result so far: [0 0 0 0 0 0 0 0]
i=2, arr[i]=4, current_max=3
  Updated: current_max=4, result[2]=4
  result so far: [0 0 4 0 0 0 0 0]
i=3, arr[i]=1, current_max=4
  result so far: [0 0 4 0 0 0 0 0]
i=4, arr[i]=5, current_max=4
  Updated: current_max=5, result[4]=5
  result so far: [0 0 4 0 5 0 0 0]
i=5, arr[i]=9, current_max=5
  Updated: current_max=9, result[5]=9
  result so far: [0 0 4 0 5 9 0 0]
i=6, arr[i]=2, current_max=9
  result so far: [0 0 4 0 5 9 0 0]
i=7, arr[i]=6, current_max=9
  result so far: [0 0 4 0 5 9 0 0]
[0 0 4 0 5 9 0 0]


The print output reveals the bug: `result[i]` is only assigned inside the `if` block. When `arr[i] <= current_max`, `result[i]` stays at 0.

Fixed version:

In [10]:
def running_max(arr):
    result = np.zeros_like(arr)
    current_max = arr[0]
    for i in range(len(arr)):
        if arr[i] > current_max:
            current_max = arr[i]
        result[i] = current_max  # Move this outside the if block
    return result

test = np.array([3, 1, 4, 1, 5, 9, 2, 6])
print(running_max(test))

[3 3 4 4 5 9 9 9]


## The Logging Module

Python's `logging` module provides a more sophisticated alternative to print debugging. It offers levels of severity, timestamps, and the ability to direct output to files without modifying your code.

### Basic Logging Setup

In [11]:
import logging

# Configure logging to show DEBUG and above
logging.basicConfig(level=logging.DEBUG)

def calculate_statistics(data):
    logging.debug(f"Input data: {data[:5]}... (length={len(data)})")

    mean = sum(data) / len(data)
    logging.info(f"Calculated mean: {mean}")

    if len(data) < 30:
        logging.warning("Small sample size may affect reliability")

    return {"mean": mean, "n": len(data)}

calculate_statistics([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

DEBUG:root:Input data: [1, 2, 3, 4, 5]... (length=10)
INFO:root:Calculated mean: 5.5


{'mean': 5.5, 'n': 10}

### Logging Levels

The logging module provides five standard levels, in order of increasing severity:

* `DEBUG` - Detailed information for diagnosing problems
* `INFO` - Confirmation that things are working
* `WARNING` - Something unexpected happened, but the program continues
* `ERROR` - A serious problem; some functionality didn't work
* `CRITICAL` - A very serious error; the program may not continue

In [16]:
import logging
import numpy as np

logging.basicConfig(level=logging.DEBUG)

def fit_model(X, y):
    logging.debug(f"Fitting model with X.shape={X.shape}")

    if X.shape[0] != len(y):
        logging.error(f"Shape mismatch: X has {X.shape[0]} rows, y has {len(y)} elements")
        raise ValueError("X and y must have same number of samples")

    if X.shape[0] < X.shape[1]:
        logging.warning("More features than samples; model may overfit")

    # ... fitting logic ...
    logging.info("Model fitted successfully")

X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([1, 2, 3])
fit_model(X, y)

DEBUG:root:Fitting model with X.shape=(3, 2)
INFO:root:Model fitted successfully


### Controlling Log Level at Runtime

A key advantage of logging is controlling verbosity without changing code:

In [ ]:
import logging

# Reset logging configuration for this example
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# During development: see everything
logging.basicConfig(level=logging.DEBUG, format="%(levelname)s: %(message)s")

logging.debug("This is a debug message")
logging.info("This is an info message")
logging.warning("This is a warning message")

# To see only warnings and above, change level to logging.WARNING

You can also control logging via environment variables or configuration files, making it easy to enable detailed logging when investigating issues.

### Logging vs Print

When to use logging instead of print:

* When you want to leave debugging code in place permanently
* When you need to control verbosity levels
* When you want timestamps or other metadata
* When you need to write output to files
* For library code that others will use (let users control logging)

When print is still appropriate:

* Quick, temporary debugging you'll remove immediately
* Simple scripts where logging setup is overkill
* Intentional program output (results, prompts)

## Interactive Debugging with pdb

Python's built-in debugger, `pdb`, lets you pause execution and interactively inspect program state. This is powerful for understanding complex bugs.

### Using breakpoint()

The simplest way to start debugging is to insert `breakpoint()` where you want to pause:

In [ ]:
import numpy as np

def calculate_correlation(x, y):
    x_centered = x - np.mean(x)
    y_centered = y - np.mean(y)

    # breakpoint()  # Uncomment to pause execution here

    numerator = np.sum(x_centered * y_centered)
    denominator = np.sqrt(np.sum(x_centered**2) * np.sum(y_centered**2))
    return numerator / denominator

x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])
print(f"Correlation: {calculate_correlation(x, y):.4f}")

When Python reaches `breakpoint()`, it drops you into an interactive prompt where you can examine variables and step through code.

### Essential pdb Commands

Once in the debugger, these commands let you explore:

**Inspecting state:**

* `p expression` - Print the value of an expression
* `pp expression` - Pretty-print (useful for complex objects)
* `l` - List source code around the current line
* `w` - Show the call stack (where you are)
* `a` - Show arguments of the current function

**Navigation:**

* `n` - Execute next line (step over function calls)
* `s` - Step into function calls
* `c` - Continue execution until next breakpoint
* `r` - Continue until current function returns
* `q` - Quit the debugger

**Breakpoints:**

* `b lineno` - Set breakpoint at line number
* `b function` - Set breakpoint at function entry
* `cl` - Clear all breakpoints

### A Debugging Session Example

Consider debugging this function:

In [19]:
import numpy as np

def weighted_least_squares(X, y, weights):
    """Fit weighted least squares: minimize sum(w_i * (y_i - X_i @ beta)^2)."""
    W = np.diag(weights)
    XtWX = X.T @ W @ X
    XtWy = X.T @ W @ y
    beta = np.linalg.solve(XtWX, XtWy)
    return beta

# Test data
X = np.array([[1, 2], [1, 3], [1, 4], [1, 5]])
y = np.array([2.1, 2.9, 4.2, 4.8])
weights = np.array([1, 1, 1, 1])

# This works
print(weighted_least_squares(X, y, weights))

# But this fails with different weights
weights_bad = np.array([1, 2, 3])  # Wrong length!
print(weighted_least_squares(X, y, weights_bad))

[0.21 0.94]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

Add a breakpoint to investigate:

In [ ]:
def weighted_least_squares(X, y, weights):
    breakpoint()  # <-- Add this
    W = np.diag(weights)
    # ...

In the debugger session:

```
> weighted_least_squares(X, y, weights_bad)
(Pdb) p X.shape
(4, 2)
(Pdb) p y.shape
(4,)
(Pdb) p weights.shape
(3,)
(Pdb) # Aha! weights has length 3, but X has 4 rows
(Pdb) n
> W = np.diag(weights)
(Pdb) n
> XtWX = X.T @ W @ X
(Pdb) p W.shape
(3, 3)
(Pdb) # W is 3x3 but X is 4x2 - matrix multiplication will fail
```

The debugger lets you see exactly where dimensions mismatch before the error occurs.

### Post-Mortem Debugging

When code crashes, you can enter the debugger at the point of failure using post-mortem debugging:

In [ ]:
import pdb

try:
    result = buggy_function()
except Exception:
    pdb.post_mortem()  # Enter debugger at the crash point

Or from the command line:

In [ ]:
%%bash
python -m pdb script.py

This runs the script under pdb, automatically entering the debugger when an exception occurs.

### Conditional Breakpoints

Sometimes you only want to break under certain conditions. You can do this programmatically:

In [ ]:
def process_items(items):
    for i, item in enumerate(items):
        if item < 0:  # Only break for negative items
            breakpoint()
        result = compute(item)

Or in pdb, set a conditional breakpoint:

```
(Pdb) b 15, item < 0   # Break at line 15 only when item < 0
```

### Question

You're debugging a function that computes the Moore-Penrose pseudoinverse. It works for most matrices but produces incorrect results for certain inputs. You've added a breakpoint to investigate.

In [ ]:
import numpy as np

def pseudoinverse(A):
    """Compute the Moore-Penrose pseudoinverse of A."""
    breakpoint()
    U, s, Vt = np.linalg.svd(A)
    # Invert non-zero singular values
    s_inv = np.where(s > 1e-10, 1/s, 0)
    # Reconstruct pseudoinverse
    return Vt.T @ np.diag(s_inv) @ U.T

A = np.array([[1, 2], [3, 4], [5, 6]])  # 3x2 matrix
result = pseudoinverse(A)

In the debugger, you run these commands:

```
(Pdb) p A.shape
(3, 2)
(Pdb) p U.shape
(3, 3)
(Pdb) p s.shape
(2,)
(Pdb) p Vt.shape
(2, 2)
(Pdb) p np.diag(s_inv).shape
(2, 2)
(Pdb) p (Vt.T @ np.diag(s_inv)).shape
(2, 2)
(Pdb) p U.T.shape
(3, 3)
```

Based on this debugging session, what is the bug?

**Answer:**

The bug is a shape mismatch in the matrix multiplication. The computation `Vt.T @ np.diag(s_inv) @ U.T` tries to multiply a (2, 2) matrix with a (3, 3) matrix, which doesn't work.

For the pseudoinverse formula A⁺ = V @ S⁺ @ U.T, the matrix S⁺ needs to have shape (2, 3) to make the multiplication valid:

* V (Vt.T) has shape (2, 2)
* S⁺ should have shape (2, 3)
* U.T has shape (3, 3)
* Result: (2, 2) @ (2, 3) @ (3, 3) → (2, 3), which is correct for A⁺

The fix is to construct S⁺ with the correct shape:

In [ ]:
def pseudoinverse(A):
    U, s, Vt = np.linalg.svd(A)
    s_inv = np.where(s > 1e-10, 1/s, 0)
    # Create S_inv with correct shape (n, m) for A of shape (m, n)
    S_inv = np.zeros((A.shape[1], A.shape[0]))
    np.fill_diagonal(S_inv, s_inv)
    return Vt.T @ S_inv @ U.T

## Debugging Strategies

Beyond specific tools, effective debugging requires systematic strategies.

### Reproduce the Bug First

Before fixing a bug, ensure you can reproduce it reliably. Create a minimal test case:

In [ ]:
# Original failing code (complex)
results = pipeline.run(large_dataset, config=complex_config)

# Minimal reproduction
def minimal_test():
    # Smallest input that triggers the bug
    data = np.array([1, 2, np.nan])
    return calculate_mean(data)

A minimal test case makes the bug easier to understand and verify that your fix works.

### Binary Search for Bug Location

When a bug appears in a large codebase, use binary search to locate it:

1. Add a check (print/breakpoint/assert) in the middle of the suspicious code
2. If the bug appears before the check, search the first half
3. If the bug appears after the check, search the second half
4. Repeat until you've isolated the problematic line

This is much faster than reading through all the code.

### Check Assumptions with Assertions

Add assertions to verify your assumptions:

In [ ]:
def fit_regression(X, y):
    assert X.ndim == 2, f"X should be 2D, got {X.ndim}D"
    assert y.ndim == 1, f"y should be 1D, got {y.ndim}D"
    assert X.shape[0] == len(y), f"X has {X.shape[0]} rows, y has {len(y)} elements"
    assert not np.any(np.isnan(X)), "X contains NaN values"
    assert not np.any(np.isnan(y)), "y contains NaN values"

    # ... implementation ...

Assertions serve as executable documentation and catch bugs early, close to their source.

### Rubber Duck Debugging

Explain your code line by line, as if to someone who knows nothing about it (or to a rubber duck). This forces you to articulate your assumptions and often reveals the bug:

"This line computes the mean... wait, I'm dividing by `n` but I should be dividing by `n-1` for the unbiased estimator."

### Compare Working and Broken Cases

When code works for some inputs but not others, compare them systematically:

In [ ]:
# Works
good_data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
result_good = my_function(good_data)

# Fails
bad_data = np.array([1.0, 2.0, 3.0, 4.0, 5.0, np.inf])
result_bad = my_function(bad_data)

# What's different?
print(f"Good: len={len(good_data)}, has_inf={np.any(np.isinf(good_data))}")
print(f"Bad: len={len(bad_data)}, has_inf={np.any(np.isinf(bad_data))}")

### Question

A colleague's function works correctly for most inputs but produces wrong results for the test case shown. Apply debugging strategies to identify the bug.

In [ ]:
import numpy as np

def standardize_columns(X):
    """Standardize each column to have mean 0 and std 1."""
    means = np.mean(X, axis=0)
    stds = np.std(X, axis=0)
    return (X - means) / stds

# Works correctly
X1 = np.array([[1, 2], [3, 4], [5, 6]])
print(np.std(standardize_columns(X1), axis=0))  # [1. 1.] - correct

# Produces incorrect results
X2 = np.array([[1, 5], [3, 5], [5, 5]])
print(np.std(standardize_columns(X2), axis=0))  # [1. nan] - wrong!

What is the bug, and how would you fix it?

**Answer:**

**Diagnosis process:**

1. Compare working and broken inputs: X1 has all different values; X2's second column is constant (all 5s).

2. Add prints to check intermediate values:
   ```python
   means = np.mean(X2, axis=0)  # [3. 5.]
   stds = np.std(X2, axis=0)    # [1.63... 0.]  <- Second std is 0!
   ```

3. The bug: dividing by `stds` when a column has zero standard deviation (constant values) causes division by zero, producing `nan` or `inf`.

**Fix:**

In [ ]:
def standardize_columns(X):
    """Standardize each column to have mean 0 and std 1."""
    means = np.mean(X, axis=0)
    stds = np.std(X, axis=0)

    # Handle constant columns (std = 0)
    stds_safe = np.where(stds == 0, 1, stds)
    result = (X - means) / stds_safe

    # Constant columns become all zeros after centering
    return result

Alternatively, raise an error or warning for constant columns:

In [ ]:
if np.any(stds == 0):
    logging.warning("Some columns have zero variance; standardization may produce NaN")

## Common Numerical Bugs

Scientific computing has specific bug patterns related to floating-point arithmetic and numerical algorithms.

### Floating-Point Comparison

Never test floating-point numbers for exact equality:

In [ ]:
# Bug: may fail due to floating-point precision
if x == 0.3:
    # ...

# Fix: use tolerance-based comparison
if abs(x - 0.3) < 1e-10:
    # ...

# Or use np.isclose for arrays
if np.isclose(x, 0.3):
    # ...

### Integer Division

In Python 3, `/` always produces float, but `//` produces integer division:

In [ ]:
print(5 / 2)
print(5 // 2)

When porting code from Python 2 or other languages, watch for unintended integer division.

### Off-by-One Errors

These are extremely common, especially with ranges and indices:

In [ ]:
# Bug: range(n) goes from 0 to n-1, not 0 to n
for i in range(n):
    print(data[i + 1])  # IndexError when i = n-1

# Fix: adjust the range
for i in range(n - 1):
    print(data[i + 1])

### NaN Propagation

NaN (Not a Number) propagates through calculations silently:

In [ ]:
x = float('nan')
print(x + 5)
print(x * 0)
print(x == x)  # NaN is not equal to itself!

Use `np.isnan()` to detect NaN values:

In [ ]:
if np.any(np.isnan(result)):
    logging.error("Computation produced NaN values")

### Overflow and Underflow

Large or small numbers can overflow to infinity or underflow to zero:

In [ ]:
import numpy as np
print(np.exp(1000))
print(np.exp(-1000))

For statistical computations, use log-space arithmetic:

In [ ]:
# Bug: may overflow
product = np.prod(probabilities)

# Fix: work in log space
log_product = np.sum(np.log(probabilities))

### Question

The following function computes the softmax, but it has a numerical stability bug. Identify the bug and fix it.

In [ ]:
import numpy as np

def softmax(x):
    """Compute softmax: exp(x_i) / sum(exp(x_j))"""
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# Works fine
print(softmax(np.array([1.0, 2.0, 3.0])))

# Produces NaN or Inf
print(softmax(np.array([1000.0, 1001.0, 1002.0])))

**Answer:**

The bug is numerical overflow. When x contains large values, `np.exp(1000)` overflows to infinity, and `inf / inf = nan`.

The fix is to subtract the maximum value before exponentiating. This doesn't change the mathematical result (it's equivalent to multiplying numerator and denominator by a constant), but it keeps numbers in a stable range:

In [ ]:
def softmax(x):
    """Compute softmax: exp(x_i) / sum(exp(x_j))"""
    # Subtract max for numerical stability
    x_shifted = x - np.max(x)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x)

Now `x_shifted` has a maximum of 0, so `exp(x_shifted)` stays in the range (0, 1], avoiding overflow.

This is a standard trick in machine learning implementations. The log-sum-exp operation uses a similar stabilization.

## Debugging in Practice

Let's walk through a realistic debugging scenario that combines multiple techniques.

### The Problem

You've written a function to fit a Gaussian mixture model using EM, but it's returning unexpected results:

In [ ]:
import numpy as np

def fit_gaussian_mixture(data, n_components=2, max_iter=100, tol=1e-6):
    """Fit a Gaussian mixture model using EM algorithm."""
    n = len(data)

    # Initialize parameters
    means = np.random.choice(data, n_components, replace=False)
    stds = np.ones(n_components) * np.std(data)
    weights = np.ones(n_components) / n_components

    for iteration in range(max_iter):
        # E-step: compute responsibilities
        resp = np.zeros((n, n_components))
        for k in range(n_components):
            resp[:, k] = weights[k] * norm_pdf(data, means[k], stds[k])
        resp = resp / resp.sum(axis=1, keepdims=True)

        # M-step: update parameters
        Nk = resp.sum(axis=0)
        weights = Nk / n
        means = (resp.T @ data) / Nk
        for k in range(n_components):
            diff = data - means[k]
            stds[k] = np.sqrt((resp[:, k] @ (diff ** 2)) / Nk[k])

        # Check convergence
        # ...

    return means, stds, weights

def norm_pdf(x, mean, std):
    """Compute normal PDF."""
    return np.exp(-0.5 * ((x - mean) / std) ** 2) / (std * np.sqrt(2 * np.pi))

Running this produces:

In [ ]:
np.random.seed(42)
data = np.concatenate([np.random.normal(0, 1, 100), np.random.normal(5, 1, 100)])
means, stds, weights = fit_gaussian_mixture(data)
print(f"Means: {means}")  # Expected: ~[0, 5], Actual: [nan, nan]

### Debugging Process

**Step 1: Add logging to track iterations**

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

for iteration in range(max_iter):
    logging.debug(f"Iteration {iteration}: means={means}, stds={stds}")
    # ...

Output shows means become NaN after a few iterations.

**Step 2: Use breakpoint to investigate the E-step**

In [ ]:
for iteration in range(max_iter):
    breakpoint()
    # E-step
    resp = np.zeros((n, n_components))
    # ...

In the debugger:

```
(Pdb) p means
array([0.12, 4.89])
(Pdb) p stds
array([1.02, 0.98])
(Pdb) c  # Continue to next iteration
(Pdb) p resp[:5]
array([[0.99, 0.01], [0.98, 0.02], ...])
(Pdb) p resp.sum(axis=1)[:5]
array([1., 1., 1., 1., 1.])  # Good, responsibilities sum to 1
```

E-step looks correct. Check M-step.

**Step 3: Check for numerical issues in M-step**

In [ ]:
# After M-step
logging.debug(f"Nk={Nk}, stds after update={stds}")

Output reveals that after several iterations, one component's Nk approaches zero, making `stds[k] = sqrt(... / 0)` produce NaN.

**Step 4: Add safeguards**

In [ ]:
# M-step: update parameters
Nk = resp.sum(axis=0)

# Prevent division by zero
Nk = np.maximum(Nk, 1e-10)

weights = Nk / n
# ...

**Step 5: Verify fix with assertions**

In [ ]:
for iteration in range(max_iter):
    # ... E-step ...

    assert not np.any(np.isnan(resp)), f"NaN in responsibilities at iter {iteration}"
    assert np.allclose(resp.sum(axis=1), 1), f"Responsibilities don't sum to 1"

    # ... M-step ...

    assert not np.any(np.isnan(means)), f"NaN in means at iter {iteration}"
    assert not np.any(np.isnan(stds)), f"NaN in stds at iter {iteration}"
    assert np.all(stds > 0), f"Non-positive std at iter {iteration}"

### Lessons from This Example

1. **Start with logging** to get an overview of what's happening
2. **Use the debugger** to inspect specific values when you have a hypothesis
3. **Check for numerical edge cases** (division by zero, overflow, underflow)
4. **Add assertions** to catch problems early and document assumptions
5. **Test the fix** to ensure it solves the problem without breaking other cases

## Recommended Resources

* [Python Debugging with pdb](https://realpython.com/python-debugging-pdb/) - Real Python's comprehensive pdb tutorial
* [Understanding the Python Traceback](https://realpython.com/python-traceback/) - How to read and understand error messages
* [Python Logging HOWTO](https://docs.python.org/3/howto/logging.html) - Official logging module documentation
* [pdb documentation](https://docs.python.org/3/library/pdb.html) - Official Python debugger reference
* [Common Python Errors](https://betterstack.com/community/guides/scaling-python/python-errors/) - Guide to common error types and fixes